In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
assert X_train.shape == (50000, 32, 32, 3)
assert X_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [ ]:
# Convert the data to TensorFlow Datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Preprocessing function
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = (image / 255.0)
    return image, label

# Dataset preparation function with augmentation option
def prepare_dataset(ds, batch_size=20):
    ds = ds.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.shuffle(buffer_size=10000).batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

# Preprocess the data
train_ds = prepare_dataset(train_ds)
test_ds = prepare_dataset(test_ds)

In [ ]:
# Define the input layer
inputs = tf.keras.layers.Input(shape=(32,32,3), name="input_layer")

# Resize the images to the size expected by ResNet
resize = tf.keras.layers.UpSampling2D(size=(7,7), name="resize_layer")(inputs)

# Load the ResNet50 model with pre-trained ImageNet weights
feature_extractor = tf.keras.applications.ResNet50(input_shape=(224, 224, 3),
                                                   include_top=False,
                                                   weights='imagenet'
                                                   )(resize)

# Add the classification layers
x = tf.keras.layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(feature_extractor)
x = tf.keras.layers.Flatten(name="flatten_layer")(x)
x = tf.keras.layers.Dense(1024, activation="relu", name="dense_layer_1", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(512, activation="relu", name="dense_layer_2", kernel_initializer="he_normal")(x)
classification_output = tf.keras.layers.Dense(10, activation="softmax", name="output_layer")(x)

# Define the model
model = tf.keras.Model(inputs=inputs, outputs=classification_output, name="my_model")

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Print the model summary
model.summary()

In [ ]:
# Print the summary of the ResNet50 model
model.layers[2].summary()

# If you want to visualize a specific layer
layer_names = [layer.name for layer in model.layers[2].layers]
print(layer_names)

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor="val_accuracy", factor=0.2, patience=3, min_lr=0.00001
)
early_stopping = EarlyStopping(
    monitor="val_accuracy", patience=5, restore_best_weights=True
)
callbacks = [early_stopping, reduce_lr]

# Train the model
history = model.fit(
    train_ds, epochs=5, validation_data=test_ds, batch_size=128, callbacks=callbacks
)

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.tight_layout()
plt.show()